In [186]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
from warnings import filterwarnings
filterwarnings('ignore')

import yfinance as yf

In [187]:
dat = yf.Ticker("BTC")
prices = dat.history(interval='5m')['Close']
prices

Datetime
2024-12-11 09:30:00-05:00    43.849998
2024-12-11 09:35:00-05:00    43.840000
2024-12-11 09:40:00-05:00    43.939899
2024-12-11 09:45:00-05:00    44.056198
2024-12-11 09:50:00-05:00    44.099998
                               ...    
2025-01-10 15:35:00-05:00    41.977001
2025-01-10 15:40:00-05:00    42.000099
2025-01-10 15:45:00-05:00    41.985001
2025-01-10 15:50:00-05:00    42.097900
2025-01-10 15:55:00-05:00    41.970001
Name: Close, Length: 1522, dtype: float64

# Klasyfikacja

Klasyfikatory można budować na mnóstwo różnych sposobów. Ogólnie trzeba się zdecydować, na ilu obserwacjach wstecz ma się opierać klasyfikacja. Liczbę tych obserwacji nazwiemy 'window' i przyjmiemy jako parametr.

Na razie ustawiamy klasyfikację na podstawie poprzednich 4 godzin.

Pierwszy pomysł to klasyfikacja na podstawie stóp zwrotu, klasy również będziemy budować na podstawie stóp zwrotu. Metoda budowania klas zostanie omówiona dalej.

In [188]:
interval = '5m' # dane 5-minutowe
returns = prices.pct_change().dropna()

window = int(4*60/int(interval[:-1]))

In [198]:
def generate_X_y(data: pd.DataFrame | pd.Series, window: int, skip: int) -> tuple:   
    # można ustawić okna zachodzące (0 < skip < window)
    # można ustawić niezachodzące (skip >= window)
    assert skip > 0, "Pętla w kodzie nigdy się nie zakończy..."
    
    # Generujemy 'okna'
    X = pd.DataFrame(columns=range(window-1))
    y = pd.Series()
    
    i = len(data)
    count = 0
    while i >= window:
        temp_y = data.iloc[i-window:i]
        
        X.loc[count, :] = temp_y.iloc[:-1].values
        y.loc[count] = temp_y.iloc[-1]
        
        i = i - skip    
        count += 1
        
    return np.array(X), np.array(y)

### Regresja logistyczna

In [179]:
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import LeaveOneOut

Pierwszy pomysł na zdefiniowanie klas to przyjęcie '1' jeśli stopa zwrotu jest $>threshold$ oraz '0' w p.p.

Próg odcięcia można podać jako parametr i już po pierwszych testach widać, że dla prostej klasyfikacji typu "czy zwrot > 0" skuteczność jest około $50\%$. Im wyższy próg, tym lepsza skuteczność.

In [ ]:
threshold = 0.0025
skip = 1

X, y = generate_X_y(data=returns, window=window, skip=skip)
y = (np.array(y) > threshold)*1.0

clf = LogisticRegression().fit(X, y)
clf.score(X, y)